In [19]:
from collections import OrderedDict
import numpy as np
import spacy
import pandas as pd
from spacy.lang.en.stop_words import STOP_WORDS

nlp = spacy.load('en_core_web_sm')

class TextRank4Keyword():
    """Extract keywords from text"""
    
    def __init__(self):
        self.d = 0.85 # damping coefficient, usually is .85
        self.min_diff = 1e-5 # convergence threshold
        self.steps = 10 # iteration steps
        self.node_weight = None # save keywords and its weight

    
    def set_stopwords(self, stopwords):  
        """Set stop words"""
        word_s = ['tha|', 'haim', 'haim|', '`', "'","’", '-', 'jaenge|', 'haim|', '"', 'hai)|', 'paenge|', '\n', '(', ')' ,'-', 'hai|','?', '!', ',', '|', '.', ';', ':', 'andar', 'ata', 'adi', 'apa', 'apana', 'apani', 'apani', 'apane', 'abhi', 'abhi',  'adi', 'apa', 'inhim', 'inhem', 'inhom', 'itayadi', 'ityadi', 'ina', 'inaka', 'inhim', 'inhem', 'inhom', 'isa', 'isaka', 'isaki', 'isaki', 'isake', 'isamem', 'isi', 'isi', 'ise', 'unhim', 'unhem', 'unhom', 'una', 'unaka', 'unaki', 'unaki', 'unake', 'unako', 'unhim', 'unhem', 'unhom', 'usa', 'usake', 'usi', 'usi', 'use', 'eka', 'evam', 'esa', 'ese', 'aise', 'ora', 'aura', 'kai', 'kai', 'kara', 'karata', 'karate', 'karana', 'karane', 'karem', 'kahate', 'kaha', 'ka', 'kaphi', 'kai', 'ki', 'kinhem', 'kinhom', 'kitana', 'kinhem', 'kinhom', 'kiya', 'kira', 'kisa', 'kisi', 'kisi', 'kise', 'ki', 'kuchha', 'kula', 'ke', 'ko', 'koi', 'koi', 'kona', 'konasa', 'kauna', 'kaunasa', 'gaya', 'ghara', 'jaba', 'jaham', 'jaham', 'ja', 'jinhem', 'jinhom', 'jitana', 'jidhara', 'jina', 'jinhem', 'jinhom', 'jisa', 'jise', 'jidhara', 'jesa', 'jese', 'jaisa', 'jaise', 'jo', 'taka', 'taba', 'taraha', 'tinhem', 'tinhom', 'tina', 'tinhem', 'tinhom', 'tisa', 'tise', 'to', 'tha', 'thi', 'thi', 'the', 'dabara', 'davara', 'diya', 'dusara', 'dusare', 'dusare', 'do', 'dvara', 'na', 'nahim', 'nahim', 'na', 'niche', 'nihayata', 'niche', 'ne', 'para', 'pahale', 'pura', 'pura', 'pe', 'phira', 'bani', 'bani', 'bahi', 'bahi', 'bahuta', 'bada', 'bala', 'bilakula', 'bhi', 'bhitara', 'bhi', 'bhitara', 'magara', 'mano', 'me', 'mem', 'yadi', 'yaha', 'yaham', 'yaham', 'yahi', 'yahi', 'ya', 'yiha', 'ye', 'rakhem', 'ravasa', 'raha', 'rahe', 'vasa', 'lie', 'liye', 'lekina', 'va', 'vageraha', 'varaga', 'varga', 'vaha', 'vaham', 'vaham', 'vahim', 'vahim', 'vale', 'vuha', 've', 'va', 'airaha', 'sanga', 'sakata', 'sakate', 'sabase', 'sabhi', 'sabhi', 'satha', 'sabuta', 'sabha', 'sara', 'se', 'so', 'hi', 'hi', 'hua', 'hua', 'hui', 'hui', 'hue', 'he', 'hem', 'hai', 'haim', 'ho', 'hota', 'hoti', 'hoti', 'hote', 'hona', 'hone', 'sabha', 'sara', 'se', 'so', 'hi', 'hi', 'hua', 'hua', 'hui', 'hui', 'hue', 'he', 'hem', 'hai', 'haim', 'ho', 'hota', 'hoti', 'hoti', 'hote', 'hona', 'hone''sabha', 'sara', 'se', 'so', 'hi', 'hi', 'hua', 'hua', 'hui', 'hui', 'hue', 'he', 'hem', 'hai', 'haim', 'ho', 'hota', 'hoti', 'hoti', 'hote', 'hona', 'hone']
       
        for word in word_s:
           lexeme = nlp.vocab[word]
           lexeme.is_stop = True 
            
    
    def sentence_segment(self, doc, candidate_pos, lower):
        """Store those words only in cadidate_pos"""
        sentences = []
        for sent in doc.sents:
            selected_words = []
            for token in sent:
                # Store words only with cadidate POS tag
                #if token.pos_ in candidate_pos and token.is_stop is False:
                if token.is_stop is False:
                    if lower is True:
                        selected_words.append(token.text.lower())
                    else:
                        selected_words.append(token.text)
            sentences.append(selected_words)
            
       # print(sentences)    
        return sentences
        
    def get_vocab(self, sentences):
        """Get all tokens"""
        vocab = OrderedDict()
        i = 0
        for sentence in sentences:
            for word in sentence:
                if word not in vocab:
                    vocab[word] = i
                    i += 1
        return vocab
    
    def get_token_pairs(self, window_size, sentences):
        """Build token_pairs from windows in sentences"""
        token_pairs = list()
        token_pairs_freq = []
        for sentence in sentences:
            for i, word in enumerate(sentence):
                for j in range(i+1, i+window_size):
                    if j >= len(sentence):
                        break
                    pair = (word, sentence[j])    
                    token_pairs.append(pair)
        #for word in token_pairs:
         #   print(word)
        unique_values = []
        unique_values = pd.Series(token_pairs).unique()
        #for i in unique_values:
            #print(i)
            
        arr = pd.DataFrame([])
       
        ls = []
        for i in unique_values:
            count = 0
            for j in token_pairs:
                if i == j:
                    count = count + 1
                else:
                    continue
            #print(i, count)
            lm = []
            lm = [i,count]
            ls.append(lm)
            #arr = arr.append(pd.DataFrame({'Pairs': pd.Series(data = [i]), 'Freq': pd.Series(data = [count])}))
        
        #for word in ls:
         #   print(word)
            
        data = pd.DataFrame(ls,columns=['Pairs','Freq'])
        data = data.sort_values('Freq', ascending = False)
        for i in data['Freq']:
            print(i)
        #for i in data['Pairs']:
         #   print(i)
        print(data)
         
      
        
        #print(token_pairs_freq)
        return token_pairs
        
    def symmetrize(self, a):
        return a + a.T - np.diag(a.diagonal())
    
    def get_matrix(self, vocab, token_pairs):
        """Get normalized matrix"""
        # Build matrix
        vocab_size = len(vocab)
        g = np.zeros((vocab_size, vocab_size), dtype='float')
        for word1, word2 in token_pairs:
            i, j = vocab[word1], vocab[word2]
            g[i][j] = 1
            
        # Get Symmeric matrix
        g = self.symmetrize(g)
        
        # Normalize matrix by column
        norm = np.sum(g, axis=0)
        g_norm = np.divide(g, norm, where=norm!=0) # this is ignore the 0 element in norm
        
        return g_norm

    
    def get_keywords(self, number):
        """Print top number keywords"""
        node_weight = OrderedDict(sorted(self.node_weight.items(), key=lambda t: t[1], reverse=True))
        for i, (key, value) in enumerate(node_weight.items()):
            print(key + ' - ' + str(value))
            if i > number:
                break
        
        
    def analyze(self, text, 
                candidate_pos=['NOUN', 'PROPN'], 
                window_size=4, lower=False, stopwords=list()):
        """Main function to analyze text"""
        
        # Set stop words
        self.set_stopwords(stopwords)
        
        # Pare text by spaCy
        doc = nlp(text)
        
        # Filter sentences
        sentences = self.sentence_segment(doc, candidate_pos, lower) # list of list of words
        
        # Build vocabulary
        vocab = self.get_vocab(sentences)
        
        # Get token_pairs from windows
        token_pairs = self.get_token_pairs(window_size, sentences)
        
        # Get normalized matrix
        g = self.get_matrix(vocab, token_pairs)
        
        # Initionlization for weight(pagerank value)
        pr = np.array([1] * len(vocab))
        
        # Iteration
        previous_pr = 0
        for epoch in range(self.steps):
            pr = (1-self.d) + self.d * np.dot(g, pr)
            if abs(previous_pr - sum(pr))  < self.min_diff:
                break
            else:
                previous_pr = sum(pr)

        # Get weight for each node
        node_weight = dict()
        for word, index in vocab.items():
            node_weight[word] = pr[index]
        
        self.node_weight = node_weight
        
        
        
text = '''shatabdi eksapresa (2 ghante), taja eksapresa (21/2ghante), intara siti eksapresa jaisi ati uttama trena sevaom ke jarie agara dilli se achchhi taraha juda़a hua hai| sansara ke paryataka manachitra para tina virasati smaraka - tajamahala, lala kila aura phatehapura sikari ke satha, agara eka pramukha gantavya sthana hai| agara kevala kabristanom tatha pattharom ka hvasonmukhi shahara hi nahim hai balki sanskriti, kala emva dharmika vicharadharaom ka aisa jivanta kendra he jinhone sadiyom se manavata ko sanriddha kiya aura manushya ke vicharom ko akriti di| agara, uttara pradesha rajya, bharata, mem sthita hai| sansara ke sata ashcharya mem se eka, taja mahala ki janmabhumi hone ki vajaha se agara mashahura hai| 16vim aura arambhika 17vim sadiyom ki rajadhani ki jiti jagati yada hai makabarom, kile aura mahalom ki sthapatya sambandhi shana| jabaki rajanitika kendra ke rupa mem agara ka mahatva 1634 mem shahajaham ke rajadhani ka sthanantarana dilli karane ke satha hi khatma ho gaya tha, lekina usaki vastu sambandhi prachurata ne antarrashtriya manachitra para usaka sthana surakshita rakha| apekshakrita dhimi raphtara vala suhavana, chhota shahara, agara, sangamaramara aura sopastona (selakhadi) para jada़ai ke shanadara kama ke lie jana jata hai jo ki usa karigara dvara kiya jata hai jo unaka vanshaja hai jinhone mugalom ke adhina kama kiya tha| agara kabhi mugala sanrajya ki rajadhani tha aura aja bhi lagata hai vaha apane atita mem ji raha hai| ashcharya nahim, ki mugala badashahom ne nirmana ke prati apane josha ki vajaha se shahara ko sansara ki kuchha uchcha koti ki imaratom se sanriddha kiya| mugala darabara ke vaibhava aura gupta sambandhom mem sadiyom ke raste chupa ke se phisala jana yaham para bahuta asana hai| agara eka purana shahara hai aura kaha jata hai ki isaka nama agravana se utpanna hai, jo ki eka vana hai jisaka ullekha virakavya / mahakavya mahabharata mem milata hai| adhunika kala mem agara pramukhata mem taba aya jaba sikandara lodi ne 1501 use apani rajadhani banaya| londiyo ke shasana ka anta bahuta jaldi hone vala tha aura agara mugalom ke kabje mem chala gaya| tisare shahanshaha akabara ke shasana kala mem agara apane purna vaibhava mem aya|
akabara ne 1565 mem vishalata/ bhavya agara phorta ke nirmana ka shriganesha kiya| agara ne apana mahatva banae rakha aura akabara ke pote shahajaham ne mugala sthapatya ki usa uttama kriti tajamahala se shahara ko sajaya aura agara phorta ke andara kai aura sundara imaratom ka nirmana kiya| tajamahala mem sangamaramara ke parde ke bichombicha mem haim mumataja mahala ka smaraka bana hai, usa para parshiyana bhasha mem kurana ke mula patha utkirna haim| makabarom ke upara eka kairina laimpa he, jisaki lau ke bare mem aisa mana jata hai ki kabhi nahim bujegi| makabarom ke upara eka kairina laimpa he, jisaki lau ke bare mem aisa mana jata hai ki kabhi nahim bujegi| makabare ke andaruni bhaga ke kendra mem haim uncha kamara, usake turanta niche eka tahakhana aura chara ashtabhujakara kamare ke konom mem hai jinaka mula uddeshya rajagharane ke anya sadasyom ki kabrom ko rakhane ke lie tha| taja mahala ke bicha mem haim shahajaham aura mumataja mahala ke smaraka| Saha Jahans cenotaph is to the left and is higher than that of his beloved which rests immediately below the dome. imarata ki dhvaniki itani shanadara hai ki gumbadadara bhitari chhata ki yojana aisi ki gai hai ki kurana ke uddharana va sangitajna ki dhunom ko pratidhvanita hom| sujava diya jata he ki apane jute punah prapta karane se pahale eka bara aura makabare ke charom ora bahara hi se ghumie aura phira usake saundarya ka ananda lijie| madhya 16vim va arambhika 17vim sadiyom mem agara atyadhika nirmana sambandhi gatividhi ka sakshi raha aura isi samaya prema ke pratika, tajamahala ka nirmana hua| madhya 16vim va arambhika 17vim sadiyom mem bani imaratem purnataya samakalina mugala shaili va atyanta uchcha gunavatta ki thi| tajamahala ka shahara aura kisi samaya mugala sanrajya ki rajadhani rahe agara mem kai smaraka haim jo mugala vastukala ki bhavyata ko darshate haim| yaha agara hi tha jaham rajavansha ke sansthapaka babara ne yamuna nadi ke tatom para pahala suvyavasthita pharasi baga lagavaya tha | usake pautra akabara ne vishala lala kile ke bhavya parakote khada़e kie, jahangira ne usaki divarom ke andara gulabi lala mahalom, pranganom aura udyanom ka nirmana kiya aura jise shahajaham ne sangamaramari masjidom, mahalom aura rangamanchom ko ratna jada़ita sapheda sangamaramara se sajaya | agara mathura ki adhika mahatvapurna bhagini nagari agravana ke rupa mem prachina mahakavya mahabharata mem ullikhita hai | agara 1475 mem badala sinha dvara basaya gaya tha aura yaha prakhyati mem taba aya jaba lodhi sanrajya ke sikandara lodhi ne yamuna se isaki samipata ke karana ise apani rajadhani ke lie chuna | babara ne lodiyom ka haraya na kevala agara para kabja karane ke lie balki mugala sanrajya ki buniyada rakhane ke lie bhi| yamuna nadi ke kinare, chandrakara mem utha hai- agara phorta| sanrata akabara dvara parikalpita emva 1565 A.D. mem nirmita yaha kila charom ora se 70 phuta unchi divara se ghira hai| agara phorta mem hai sundara parla moska (moti masjida) tatha aneka mahala jinamem shamila hai- jahangiri mahala, khasa mahala, divani khasa, divani ama, machhihi bhavana aura moti masjida| agara phorta ke chara dvara hai aura charom ora se lala balua patthara(senda stona) ki duguni parakotedara divara se ghira hai| tajamahala ke donom tarapha ki tajamahala ki masjidem lala balua patthara ki imaratem hai| masjida ko dekhane se pahale, aie dhyana dem pashchima sima ki divara ki eka chhote se patthara ki gherabandi para, jaham masjida ka kuma sthita hai| yaha hariyali dara chhamva se dhaka 19*6.5 phuta ka dhancha usa sthana ki nishani hai jaham pahali bara agara lane para mumataja mahala ke avashesha rakhe gaye the| masjida ke samane ka chabutara reda senda stona ( lala balua patthara) ka bana hai| eka bahuta jyada polishadara chhota sangamaramara ka tukada़a isa taraha se phita kiya gaya hai ki vaha aine ka kama karata hai aura koi bhi makabare ko usame pratibimbita dekha sakata hai| masjida ki chhata chara ashta bhujakara minarom aura tina bhavya gumbadom ko sahara deti hai| taja mahala ki kahani mumataja़ ke jivana se abhinna hai jo shahajaham ki patarani thi| mugala badashaha banane se pahale shahaja़ada kha़urrama ke nama se jana jane vala shahajaham, bisa varsha ka eka sundara purusha tha, jaba usaki mumataja se sagai hui| aisa kaha jata hai ki agara ki shahi rajadhani sagai ke samaya mumataja ki sundarata ke varnana ke karana utsukata se bhari hui thi| mumataja mahala ka vivaha ke purva ka nama arjumanda bano bega़ma tha| shahajada khurrama mumataja mahala se milane aura vivaha karane se purva do bara shadi kara chuke the |
aisa kaha jata hai ki mumataja mahala ne apani nrityu shayya para kha़uda apane shauhara ko bhavi pidha़i ke lie unake pyara ke eka pratika ki sarjana karane ke lie kaha tha | tajamahala ka nirmana prarambha hone se purva, shavotkhanana aura antima dapha़na ke lie agara lane se pahale, mumataja mahala ko burahanapura ke ja़ainabadi baga़ mem chhah mahine ke lie asthayi rupa se dapha़naya gaya tha| tajamahala prema ka eka smaraka hai| taja bharata ka gaurava hai| tajamahala ke pichhe ki kahani bhi utani hi sundara hai jitana ki yaha smaraka| arjumanda bano, eka dukanadara, harama se sate ga़airasarakari baja़ara, mina baja़ara mem apani dukana para baithi thi, jaba shahajada khurrama ne use pahali bara dekha| khurrama ne arjumanda bano ki dukana mem eka kancha ka tukada़a dekha aura usaka mulya puchha to usane javaba diya ki yaha hira hai kancha nahim | badashaha jahangira ne shahajada khurrama ko ‘shahajaham’ ki upadhi di aura jaba vaha badashaha bana taba usane apani patni ko mumataja mahala ‘mahala ki eka sarvapriya’ ki upadhi di. mumataja़ bahuta dayalu, udara aura saunya thi| marane ke pahale mumataja़ ne shahajaham se eka vada karavaya ki usaki nrityu ke bada vaha apane pyara ki shraddhanjali ke rupa mem sansara ka sabase sundara bhavana banavaega|
sikandara kila shahanshaha akabara ka makabara hai, jo ki eka utkrishta kshetra mem sarvottama hindu aura musalamani vastukala ko mishrita karate hue usake siddhanta aura dharmanirapeksha darshana ka pratinidhitva karata hai| usake putra jahangira dvara 1613 isavi mem pura kiya gaya sikandara kila achchhi taraha se surakshita makabarom mem se eka hai| sikandara kila mugala shahanshaha akabara ki akhiri aramagaha hai|
akabara ke smaraka ka darshana usake vyaktitva ki sampurnata ko puri taraha se logom ke samane vaise hi pesha karata hai jaise taja़mahala mumataja़ mumataja़ mahala ke vyaktitva ka karata hai| sundara tarike se nakkashi ki hui, lala gerue ranga ke balue patthara ki akabara ki vishala kabra saghana bagiche ke bicha mem sthita hai| apane hi jivana kala mem eka makabare ka nirmana karana eka tatari rivaja tha jise mugalom ne nishthapurvaka apanaya| lala balue patthara ki bani minarem bahubhuji akritiyom ke sapheda sangamaramara se jada़ita haim; yaha manohara anupata aura brihata satahi sajavata praveshadvarom ko bahuta hi bhavya banati hai| praveshadvara hindu, muslima, isai vastukala ki vibhinna shailiyom aura akabara ki vishishta shaili ki uchcha koti ka eka durlabha mishrana pratibimbita karata hai| eka chauda़a pakka marga kabra taka jata hai, jo pancha manjilom ki hai aura chhinnabha piramida ke akara mem hai| mukhya kabra eka khasa chaukora akriti ki hai jo dusari mugala imaratom se bejoda़ hai| dakshina bhaga ke kendra mem galiyare haim jise bahuta adhika matra mem utkrishta nakkashi, kalatmaka chitrakari tatha jyamitiya aura phuladara akritiyom mem jada़au kala dvara sajaya gaya hai| akabara ki kabra ka patthara kamare ke bicha mem sthita hai| atyadhika chamakai gai tailom ya ranga-birange pattharom se ki pachchikari (mojaika) ke dvara jyamitiya akritiyam prapta ki gai haim jinaki kabra para pradhanata hai| eka sajavati meharaba aura chaukora stambha ke upayoga se anuthi sanrachana bani hai| panchavi manjila puri taraha se sapheda sangamaramara ki hai jabaki nichali manjila lala balue patthara ki bani hai| banavati praveshadvarom ko, jinhem uchita rupa se jala khema kaha ja sakata hai, dakshini aura uttari kinarom ke kendra mem banaya gaya tha| uthale pani ki naharem, jo arambha mem nadi kinare bani do upari tankiyom se pani leti thim, bagiche ke sabhi ora tatha makabare ke charom ora bahati haim| taja़mahala ki kahani prema ki utkatata ko pradarshita karati hai| akabara ne sansara-bhara se vastu shilpiyom aura karigarom ko amantrita kiya aura mahala-nirmana ke lie puri nipunata se yojana banai|
tajamahala aphaganistana se bulae gae shilpakarom ki nakkashi aura kashmira ke udyana rachanakarom ke parasi shaili mem sanyojana se banaya gaya tha| tajamahala ki nakkashi anamola ratnom se alankrita ki gayi thi| manava jivana mem bhavanaom aura samvedanaom ka kitana mahatva hai, tajamahala isaka eka pramana hai| tajamahala ki kahani samarpana aura vishvasa ka eka udaharana hai| tajamahala ki kahani eka prema katha hai jo ki kagaja़om para nahim milati balki sanrachanatmaka rupa mem khada़i hai| baramade ki chhata peda़om, chida़iyom, pashuom aura jyamitiya namunom se chitrita hai| dvaramarga para stri sanrakshakom, jala hansom, bela- butom aura kamaladalom se ghire hue atha buddhom ki eka katara hai| vistrita nakkashi kiya hua antarika bhaga sabase utkrishta nishpadita chattana shilpa chaityom mem se eka hai| bamim divara para chitrakari ke eka tukada़e mem buddha eka sabha ko pravachana de rahe haim| baramade ke kinare ke punya sthalom ke upara sanriddhi ki devi hariti aura daim ora unake sahavarti aura bamim ora sarparaja naga ki eka sabha utkirnita hai| adhika uchcha stara para khude hue, eka sankare baramade vale isa chhote se matha mem chara apurna kaksha haim| 
matha ki bamim divara para apane-apane bodhi vrikshom ke niche maitreya ke satha sata manushi buddha chitrita haim|
isa chaitya ke sabhagriha ka antarika bhaga utkirnita buddha chitrom se alankrita hai| kendriya stupa mem mandapa mem baithe guru ki eka pratima hai| bamim divara para do vivaranatmaka drishya anuraga ke devata mara ke dvara pralobhana ko darshate haim, aura lete hue buddha ki vishalakaya utkirnita akriti unake punarjanma ke chakra se antima moksha ki prapti, mahaparinirvana, ko darshati hai| ajanta ke prachina kalakarom ne khuraduri divarom ko puala ya janavarom ke bala mili gili mitti aura gobara ki parata rupi yojaka madhyama se lepa hai| ajanta guphaom ki chitrakari ko praya: bhramavasha bhittichitra kaha jata hai| bhittichitra gile chune ke palastara para rangom ke lagae jane ko kahate hai| ajanta mem ye chitrakariyam sukhi divara para ki gayi thim|
eka kalpanika siddhanta ke anusara mukhya koshtha ki jamina para gaddhe, jo ki sabhi guphaom ki eka vishishtata haim, pani se bhare the, jinamem andhere se ho kara kuchha kiranem pahuncha pati thim| ajanta ki chitrakari ka eka aura ashcharyajanaka tathya yaha hai ki ve gupha mem bahuta hi manda prakasha mem ki gayi thim|
eka chhora para baingalora ka navina audyogika shahara tatha dusare chhora para dehata mem kheti ke vrihad kshetra ke satha purva mem maisura ke nama se prasiddha, karnataka rajya atyadhika vibhinnataom ka rajya hai| bharata ke kuchha sabase ruchikara aitihasika sthapatya bhi karnataka mem haim| bagom ke shahara mem apa lala baga mem ghuma sakate haim, jisamem kai prakara ke purane vriksha, phavvare, kamala tala, khule maidana tatha nanaprakara ke ushna katibandhiya tatha upa ushnakatibandhiya jada़i butiyom ke sangraha haim| bagom ke shahara mem apa maharaja ke mahala tatha sarakari imaratom ka bhi bhramana kara sakate haim|
'''
tr4w = TextRank4Keyword()
tr4w.analyze(text, candidate_pos = ['NOUN', 'PROPN'], window_size=2, lower=False)
tr4w.get_keywords(25)

8
4
3
3
3
3
3
3
3
3
3
3
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [20]:
from collections import OrderedDict
import numpy as np
import spacy
import pandas as pd
from spacy.lang.en.stop_words import STOP_WORDS

nlp = spacy.load('en_core_web_sm')

class TextRank4Keyword():
    """Extract keywords from text"""
    
    def __init__(self):
        self.d = 0.85 # damping coefficient, usually is .85
        self.min_diff = 1e-5 # convergence threshold
        self.steps = 10 # iteration steps
        self.node_weight = None # save keywords and its weight

    
    def set_stopwords(self, stopwords):  
        """Set stop words"""
        word_s =['tha|', 'haim', 'haim|', '`', "'","’", '-', 'jaenge|', 'haim|', '"', 'hai)|', 'paenge|', '\n', '(', ')' ,'-', 'hai|','?', '!', ',', '|', '.', ';', ':', 'andar', 'ata', 'adi', 'apa', 'apana', 'apani', 'apani', 'apane', 'abhi', 'abhi',  'adi', 'apa', 'inhim', 'inhem', 'inhom', 'itayadi', 'ityadi', 'ina', 'inaka', 'inhim', 'inhem', 'inhom', 'isa', 'isaka', 'isaki', 'isaki', 'isake', 'isamem', 'isi', 'isi', 'ise', 'unhim', 'unhem', 'unhom', 'una', 'unaka', 'unaki', 'unaki', 'unake', 'unako', 'unhim', 'unhem', 'unhom', 'usa', 'usake', 'usi', 'usi', 'use', 'eka', 'evam', 'esa', 'ese', 'aise', 'ora', 'aura', 'kai', 'kai', 'kara', 'karata', 'karate', 'karana', 'karane', 'karem', 'kahate', 'kaha', 'ka', 'kaphi', 'kai', 'ki', 'kinhem', 'kinhom', 'kitana', 'kinhem', 'kinhom', 'kiya', 'kira', 'kisa', 'kisi', 'kisi', 'kise', 'ki', 'kuchha', 'kula', 'ke', 'ko', 'koi', 'koi', 'kona', 'konasa', 'kauna', 'kaunasa', 'gaya', 'ghara', 'jaba', 'jaham', 'jaham', 'ja', 'jinhem', 'jinhom', 'jitana', 'jidhara', 'jina', 'jinhem', 'jinhom', 'jisa', 'jise', 'jidhara', 'jesa', 'jese', 'jaisa', 'jaise', 'jo', 'taka', 'taba', 'taraha', 'tinhem', 'tinhom', 'tina', 'tinhem', 'tinhom', 'tisa', 'tise', 'to', 'tha', 'thi', 'thi', 'the', 'dabara', 'davara', 'diya', 'dusara', 'dusare', 'dusare', 'do', 'dvara', 'na', 'nahim', 'nahim', 'na', 'niche', 'nihayata', 'niche', 'ne', 'para', 'pahale', 'pura', 'pura', 'pe', 'phira', 'bani', 'bani', 'bahi', 'bahi', 'bahuta', 'bada', 'bala', 'bilakula', 'bhi', 'bhitara', 'bhi', 'bhitara', 'magara', 'mano', 'me', 'mem', 'yadi', 'yaha', 'yaham', 'yaham', 'yahi', 'yahi', 'ya', 'yiha', 'ye', 'rakhem', 'ravasa', 'raha', 'rahe', 'vasa', 'lie', 'liye', 'lekina', 'va', 'vageraha', 'varaga', 'varga', 'vaha', 'vaham', 'vaham', 'vahim', 'vahim', 'vale', 'vuha', 've', 'va', 'airaha', 'sanga', 'sakata', 'sakate', 'sabase', 'sabhi', 'sabhi', 'satha', 'sabuta', 'sabha', 'sara', 'se', 'so', 'hi', 'hi', 'hua', 'hua', 'hui', 'hui', 'hue', 'he', 'hem', 'hai', 'haim', 'ho', 'hota', 'hoti', 'hoti', 'hote', 'hona', 'hone', 'sabha', 'sara', 'se', 'so', 'hi', 'hi', 'hua', 'hua', 'hui', 'hui', 'hue', 'he', 'hem', 'hai', 'haim', 'ho', 'hota', 'hoti', 'hoti', 'hote', 'hona', 'hone''sabha', 'sara', 'se', 'so', 'hi', 'hi', 'hua', 'hua', 'hui', 'hui', 'hue', 'he', 'hem', 'hai', 'haim', 'ho', 'hota', 'hoti', 'hoti', 'hote', 'hona', 'hone']
       
        for word in word_s:
           lexeme = nlp.vocab[word]
           lexeme.is_stop = True 
            
    
    def sentence_segment(self, doc, candidate_pos, lower):
        """Store those words only in cadidate_pos"""
        sentences = []
        for sent in doc.sents:
            selected_words = []
            for token in sent:
                # Store words only with cadidate POS tag
                #if token.pos_ in candidate_pos and token.is_stop is False:
                if token.is_stop is False:
                    if lower is True:
                        selected_words.append(token.text.lower())
                    else:
                        selected_words.append(token.text)
            sentences.append(selected_words)
            
       # print(sentences)    
        return sentences
        
    def get_vocab(self, sentences):
        """Get all tokens"""
        vocab = OrderedDict()
        i = 0
        for sentence in sentences:
            for word in sentence:
                if word not in vocab:
                    vocab[word] = i
                    i += 1
        return vocab
    
    def get_token_pairs(self, window_size, sentences):
        """Build token_pairs from windows in sentences"""
        token_pairs = list()
        token_pairs_freq = []
        for sentence in sentences:
            for i, word in enumerate(sentence):
                for j in range(i+1, i+window_size):
                    if j >= len(sentence):
                        break
                    pair = (word, sentence[j])    
                    token_pairs.append(pair)
        #for word in token_pairs:
         #   print(word)
        unique_values = []
        unique_values = pd.Series(token_pairs).unique()
        #for i in unique_values:
            #print(i)
            
        arr = pd.DataFrame([])
       
        ls = []
        for i in unique_values:
            count = 0
            for j in token_pairs:
                if i == j:
                    count = count + 1
                else:
                    continue
            #print(i, count)
            lm = []
            lm = [i,count]
            ls.append(lm)
            #arr = arr.append(pd.DataFrame({'Pairs': pd.Series(data = [i]), 'Freq': pd.Series(data = [count])}))
        
        #for word in ls:
         #   print(word)
            
        data = pd.DataFrame(ls,columns=['Pairs','Freq'])
        data = data.sort_values('Freq', ascending = False)
        #for i in data['Freq']:
         #   print(i)
        for i in data['Pairs']:
            print(i)
        print(data)
         
      
        
        #print(token_pairs_freq)
        return token_pairs
        
    def symmetrize(self, a):
        return a + a.T - np.diag(a.diagonal())
    
    def get_matrix(self, vocab, token_pairs):
        """Get normalized matrix"""
        # Build matrix
        vocab_size = len(vocab)
        g = np.zeros((vocab_size, vocab_size), dtype='float')
        for word1, word2 in token_pairs:
            i, j = vocab[word1], vocab[word2]
            g[i][j] = 1
            
        # Get Symmeric matrix
        g = self.symmetrize(g)
        
        # Normalize matrix by column
        norm = np.sum(g, axis=0)
        g_norm = np.divide(g, norm, where=norm!=0) # this is ignore the 0 element in norm
        
        return g_norm

    
    def get_keywords(self, number):
        """Print top number keywords"""
        node_weight = OrderedDict(sorted(self.node_weight.items(), key=lambda t: t[1], reverse=True))
        for i, (key, value) in enumerate(node_weight.items()):
            print(key + ' - ' + str(value))
            if i > number:
                break
        
        
    def analyze(self, text, 
                candidate_pos=['NOUN', 'PROPN'], 
                window_size=4, lower=False, stopwords=list()):
        """Main function to analyze text"""
        
        # Set stop words
        self.set_stopwords(stopwords)
        
        # Pare text by spaCy
        doc = nlp(text)
        
        # Filter sentences
        sentences = self.sentence_segment(doc, candidate_pos, lower) # list of list of words
        
        # Build vocabulary
        vocab = self.get_vocab(sentences)
        
        # Get token_pairs from windows
        token_pairs = self.get_token_pairs(window_size, sentences)
        
        # Get normalized matrix
        g = self.get_matrix(vocab, token_pairs)
        
        # Initionlization for weight(pagerank value)
        pr = np.array([1] * len(vocab))
        
        # Iteration
        previous_pr = 0
        for epoch in range(self.steps):
            pr = (1-self.d) + self.d * np.dot(g, pr)
            if abs(previous_pr - sum(pr))  < self.min_diff:
                break
            else:
                previous_pr = sum(pr)

        # Get weight for each node
        node_weight = dict()
        for word, index in vocab.items():
            node_weight[word] = pr[index]
        
        self.node_weight = node_weight
        
        
        
text = '''shatabdi eksapresa (2 ghante), taja eksapresa (21/2ghante), intara siti eksapresa jaisi ati uttama trena sevaom ke jarie agara dilli se achchhi taraha juda़a hua hai| sansara ke paryataka manachitra para tina virasati smaraka - tajamahala, lala kila aura phatehapura sikari ke satha, agara eka pramukha gantavya sthana hai| agara kevala kabristanom tatha pattharom ka hvasonmukhi shahara hi nahim hai balki sanskriti, kala emva dharmika vicharadharaom ka aisa jivanta kendra he jinhone sadiyom se manavata ko sanriddha kiya aura manushya ke vicharom ko akriti di| agara, uttara pradesha rajya, bharata, mem sthita hai| sansara ke sata ashcharya mem se eka, taja mahala ki janmabhumi hone ki vajaha se agara mashahura hai| 16vim aura arambhika 17vim sadiyom ki rajadhani ki jiti jagati yada hai makabarom, kile aura mahalom ki sthapatya sambandhi shana| jabaki rajanitika kendra ke rupa mem agara ka mahatva 1634 mem shahajaham ke rajadhani ka sthanantarana dilli karane ke satha hi khatma ho gaya tha, lekina usaki vastu sambandhi prachurata ne antarrashtriya manachitra para usaka sthana surakshita rakha| apekshakrita dhimi raphtara vala suhavana, chhota shahara, agara, sangamaramara aura sopastona (selakhadi) para jada़ai ke shanadara kama ke lie jana jata hai jo ki usa karigara dvara kiya jata hai jo unaka vanshaja hai jinhone mugalom ke adhina kama kiya tha| agara kabhi mugala sanrajya ki rajadhani tha aura aja bhi lagata hai vaha apane atita mem ji raha hai| ashcharya nahim, ki mugala badashahom ne nirmana ke prati apane josha ki vajaha se shahara ko sansara ki kuchha uchcha koti ki imaratom se sanriddha kiya| mugala darabara ke vaibhava aura gupta sambandhom mem sadiyom ke raste chupa ke se phisala jana yaham para bahuta asana hai| agara eka purana shahara hai aura kaha jata hai ki isaka nama agravana se utpanna hai, jo ki eka vana hai jisaka ullekha virakavya / mahakavya mahabharata mem milata hai| adhunika kala mem agara pramukhata mem taba aya jaba sikandara lodi ne 1501 use apani rajadhani banaya| londiyo ke shasana ka anta bahuta jaldi hone vala tha aura agara mugalom ke kabje mem chala gaya| tisare shahanshaha akabara ke shasana kala mem agara apane purna vaibhava mem aya|
akabara ne 1565 mem vishalata/ bhavya agara phorta ke nirmana ka shriganesha kiya| agara ne apana mahatva banae rakha aura akabara ke pote shahajaham ne mugala sthapatya ki usa uttama kriti tajamahala se shahara ko sajaya aura agara phorta ke andara kai aura sundara imaratom ka nirmana kiya| tajamahala mem sangamaramara ke parde ke bichombicha mem haim mumataja mahala ka smaraka bana hai, usa para parshiyana bhasha mem kurana ke mula patha utkirna haim| makabarom ke upara eka kairina laimpa he, jisaki lau ke bare mem aisa mana jata hai ki kabhi nahim bujegi| makabarom ke upara eka kairina laimpa he, jisaki lau ke bare mem aisa mana jata hai ki kabhi nahim bujegi| makabare ke andaruni bhaga ke kendra mem haim uncha kamara, usake turanta niche eka tahakhana aura chara ashtabhujakara kamare ke konom mem hai jinaka mula uddeshya rajagharane ke anya sadasyom ki kabrom ko rakhane ke lie tha| taja mahala ke bicha mem haim shahajaham aura mumataja mahala ke smaraka| Saha Jahans cenotaph is to the left and is higher than that of his beloved which rests immediately below the dome. imarata ki dhvaniki itani shanadara hai ki gumbadadara bhitari chhata ki yojana aisi ki gai hai ki kurana ke uddharana va sangitajna ki dhunom ko pratidhvanita hom| sujava diya jata he ki apane jute punah prapta karane se pahale eka bara aura makabare ke charom ora bahara hi se ghumie aura phira usake saundarya ka ananda lijie| madhya 16vim va arambhika 17vim sadiyom mem agara atyadhika nirmana sambandhi gatividhi ka sakshi raha aura isi samaya prema ke pratika, tajamahala ka nirmana hua| madhya 16vim va arambhika 17vim sadiyom mem bani imaratem purnataya samakalina mugala shaili va atyanta uchcha gunavatta ki thi| tajamahala ka shahara aura kisi samaya mugala sanrajya ki rajadhani rahe agara mem kai smaraka haim jo mugala vastukala ki bhavyata ko darshate haim| yaha agara hi tha jaham rajavansha ke sansthapaka babara ne yamuna nadi ke tatom para pahala suvyavasthita pharasi baga lagavaya tha | usake pautra akabara ne vishala lala kile ke bhavya parakote khada़e kie, jahangira ne usaki divarom ke andara gulabi lala mahalom, pranganom aura udyanom ka nirmana kiya aura jise shahajaham ne sangamaramari masjidom, mahalom aura rangamanchom ko ratna jada़ita sapheda sangamaramara se sajaya | agara mathura ki adhika mahatvapurna bhagini nagari agravana ke rupa mem prachina mahakavya mahabharata mem ullikhita hai | agara 1475 mem badala sinha dvara basaya gaya tha aura yaha prakhyati mem taba aya jaba lodhi sanrajya ke sikandara lodhi ne yamuna se isaki samipata ke karana ise apani rajadhani ke lie chuna | babara ne lodiyom ka haraya na kevala agara para kabja karane ke lie balki mugala sanrajya ki buniyada rakhane ke lie bhi| yamuna nadi ke kinare, chandrakara mem utha hai- agara phorta| sanrata akabara dvara parikalpita emva 1565 A.D. mem nirmita yaha kila charom ora se 70 phuta unchi divara se ghira hai| agara phorta mem hai sundara parla moska (moti masjida) tatha aneka mahala jinamem shamila hai- jahangiri mahala, khasa mahala, divani khasa, divani ama, machhihi bhavana aura moti masjida| agara phorta ke chara dvara hai aura charom ora se lala balua patthara(senda stona) ki duguni parakotedara divara se ghira hai| tajamahala ke donom tarapha ki tajamahala ki masjidem lala balua patthara ki imaratem hai| masjida ko dekhane se pahale, aie dhyana dem pashchima sima ki divara ki eka chhote se patthara ki gherabandi para, jaham masjida ka kuma sthita hai| yaha hariyali dara chhamva se dhaka 19*6.5 phuta ka dhancha usa sthana ki nishani hai jaham pahali bara agara lane para mumataja mahala ke avashesha rakhe gaye the| masjida ke samane ka chabutara reda senda stona ( lala balua patthara) ka bana hai| eka bahuta jyada polishadara chhota sangamaramara ka tukada़a isa taraha se phita kiya gaya hai ki vaha aine ka kama karata hai aura koi bhi makabare ko usame pratibimbita dekha sakata hai| masjida ki chhata chara ashta bhujakara minarom aura tina bhavya gumbadom ko sahara deti hai| taja mahala ki kahani mumataja़ ke jivana se abhinna hai jo shahajaham ki patarani thi| mugala badashaha banane se pahale shahaja़ada kha़urrama ke nama se jana jane vala shahajaham, bisa varsha ka eka sundara purusha tha, jaba usaki mumataja se sagai hui| aisa kaha jata hai ki agara ki shahi rajadhani sagai ke samaya mumataja ki sundarata ke varnana ke karana utsukata se bhari hui thi| mumataja mahala ka vivaha ke purva ka nama arjumanda bano bega़ma tha| shahajada khurrama mumataja mahala se milane aura vivaha karane se purva do bara shadi kara chuke the |
aisa kaha jata hai ki mumataja mahala ne apani nrityu shayya para kha़uda apane shauhara ko bhavi pidha़i ke lie unake pyara ke eka pratika ki sarjana karane ke lie kaha tha | tajamahala ka nirmana prarambha hone se purva, shavotkhanana aura antima dapha़na ke lie agara lane se pahale, mumataja mahala ko burahanapura ke ja़ainabadi baga़ mem chhah mahine ke lie asthayi rupa se dapha़naya gaya tha| tajamahala prema ka eka smaraka hai| taja bharata ka gaurava hai| tajamahala ke pichhe ki kahani bhi utani hi sundara hai jitana ki yaha smaraka| arjumanda bano, eka dukanadara, harama se sate ga़airasarakari baja़ara, mina baja़ara mem apani dukana para baithi thi, jaba shahajada khurrama ne use pahali bara dekha| khurrama ne arjumanda bano ki dukana mem eka kancha ka tukada़a dekha aura usaka mulya puchha to usane javaba diya ki yaha hira hai kancha nahim | badashaha jahangira ne shahajada khurrama ko ‘shahajaham’ ki upadhi di aura jaba vaha badashaha bana taba usane apani patni ko mumataja mahala ‘mahala ki eka sarvapriya’ ki upadhi di. mumataja़ bahuta dayalu, udara aura saunya thi| marane ke pahale mumataja़ ne shahajaham se eka vada karavaya ki usaki nrityu ke bada vaha apane pyara ki shraddhanjali ke rupa mem sansara ka sabase sundara bhavana banavaega|
sikandara kila shahanshaha akabara ka makabara hai, jo ki eka utkrishta kshetra mem sarvottama hindu aura musalamani vastukala ko mishrita karate hue usake siddhanta aura dharmanirapeksha darshana ka pratinidhitva karata hai| usake putra jahangira dvara 1613 isavi mem pura kiya gaya sikandara kila achchhi taraha se surakshita makabarom mem se eka hai| sikandara kila mugala shahanshaha akabara ki akhiri aramagaha hai|
akabara ke smaraka ka darshana usake vyaktitva ki sampurnata ko puri taraha se logom ke samane vaise hi pesha karata hai jaise taja़mahala mumataja़ mumataja़ mahala ke vyaktitva ka karata hai| sundara tarike se nakkashi ki hui, lala gerue ranga ke balue patthara ki akabara ki vishala kabra saghana bagiche ke bicha mem sthita hai| apane hi jivana kala mem eka makabare ka nirmana karana eka tatari rivaja tha jise mugalom ne nishthapurvaka apanaya| lala balue patthara ki bani minarem bahubhuji akritiyom ke sapheda sangamaramara se jada़ita haim; yaha manohara anupata aura brihata satahi sajavata praveshadvarom ko bahuta hi bhavya banati hai| praveshadvara hindu, muslima, isai vastukala ki vibhinna shailiyom aura akabara ki vishishta shaili ki uchcha koti ka eka durlabha mishrana pratibimbita karata hai| eka chauda़a pakka marga kabra taka jata hai, jo pancha manjilom ki hai aura chhinnabha piramida ke akara mem hai| mukhya kabra eka khasa chaukora akriti ki hai jo dusari mugala imaratom se bejoda़ hai| dakshina bhaga ke kendra mem galiyare haim jise bahuta adhika matra mem utkrishta nakkashi, kalatmaka chitrakari tatha jyamitiya aura phuladara akritiyom mem jada़au kala dvara sajaya gaya hai| akabara ki kabra ka patthara kamare ke bicha mem sthita hai| atyadhika chamakai gai tailom ya ranga-birange pattharom se ki pachchikari (mojaika) ke dvara jyamitiya akritiyam prapta ki gai haim jinaki kabra para pradhanata hai| eka sajavati meharaba aura chaukora stambha ke upayoga se anuthi sanrachana bani hai| panchavi manjila puri taraha se sapheda sangamaramara ki hai jabaki nichali manjila lala balue patthara ki bani hai| banavati praveshadvarom ko, jinhem uchita rupa se jala khema kaha ja sakata hai, dakshini aura uttari kinarom ke kendra mem banaya gaya tha| uthale pani ki naharem, jo arambha mem nadi kinare bani do upari tankiyom se pani leti thim, bagiche ke sabhi ora tatha makabare ke charom ora bahati haim| taja़mahala ki kahani prema ki utkatata ko pradarshita karati hai| akabara ne sansara-bhara se vastu shilpiyom aura karigarom ko amantrita kiya aura mahala-nirmana ke lie puri nipunata se yojana banai|
tajamahala aphaganistana se bulae gae shilpakarom ki nakkashi aura kashmira ke udyana rachanakarom ke parasi shaili mem sanyojana se banaya gaya tha| tajamahala ki nakkashi anamola ratnom se alankrita ki gayi thi| manava jivana mem bhavanaom aura samvedanaom ka kitana mahatva hai, tajamahala isaka eka pramana hai| tajamahala ki kahani samarpana aura vishvasa ka eka udaharana hai| tajamahala ki kahani eka prema katha hai jo ki kagaja़om para nahim milati balki sanrachanatmaka rupa mem khada़i hai| baramade ki chhata peda़om, chida़iyom, pashuom aura jyamitiya namunom se chitrita hai| dvaramarga para stri sanrakshakom, jala hansom, bela- butom aura kamaladalom se ghire hue atha buddhom ki eka katara hai| vistrita nakkashi kiya hua antarika bhaga sabase utkrishta nishpadita chattana shilpa chaityom mem se eka hai| bamim divara para chitrakari ke eka tukada़e mem buddha eka sabha ko pravachana de rahe haim| baramade ke kinare ke punya sthalom ke upara sanriddhi ki devi hariti aura daim ora unake sahavarti aura bamim ora sarparaja naga ki eka sabha utkirnita hai| adhika uchcha stara para khude hue, eka sankare baramade vale isa chhote se matha mem chara apurna kaksha haim| 
matha ki bamim divara para apane-apane bodhi vrikshom ke niche maitreya ke satha sata manushi buddha chitrita haim|
isa chaitya ke sabhagriha ka antarika bhaga utkirnita buddha chitrom se alankrita hai| kendriya stupa mem mandapa mem baithe guru ki eka pratima hai| bamim divara para do vivaranatmaka drishya anuraga ke devata mara ke dvara pralobhana ko darshate haim, aura lete hue buddha ki vishalakaya utkirnita akriti unake punarjanma ke chakra se antima moksha ki prapti, mahaparinirvana, ko darshati hai| ajanta ke prachina kalakarom ne khuraduri divarom ko puala ya janavarom ke bala mili gili mitti aura gobara ki parata rupi yojaka madhyama se lepa hai| ajanta guphaom ki chitrakari ko praya: bhramavasha bhittichitra kaha jata hai| bhittichitra gile chune ke palastara para rangom ke lagae jane ko kahate hai| ajanta mem ye chitrakariyam sukhi divara para ki gayi thim|
eka kalpanika siddhanta ke anusara mukhya koshtha ki jamina para gaddhe, jo ki sabhi guphaom ki eka vishishtata haim, pani se bhare the, jinamem andhere se ho kara kuchha kiranem pahuncha pati thim| ajanta ki chitrakari ka eka aura ashcharyajanaka tathya yaha hai ki ve gupha mem bahuta hi manda prakasha mem ki gayi thim|
eka chhora para baingalora ka navina audyogika shahara tatha dusare chhora para dehata mem kheti ke vrihad kshetra ke satha purva mem maisura ke nama se prasiddha, karnataka rajya atyadhika vibhinnataom ka rajya hai| bharata ke kuchha sabase ruchikara aitihasika sthapatya bhi karnataka mem haim| bagom ke shahara mem apa lala baga mem ghuma sakate haim, jisamem kai prakara ke purane vriksha, phavvare, kamala tala, khule maidana tatha nanaprakara ke ushna katibandhiya tatha upa ushnakatibandhiya jada़i butiyom ke sangraha haim| bagom ke shahara mem apa maharaja ke mahala tatha sarakari imaratom ka bhi bhramana kara sakate haim|
'''
tr4w = TextRank4Keyword()
tr4w.analyze(text, candidate_pos = ['NOUN', 'PROPN'], window_size=2, lower=False)
tr4w.get_keywords(25)

('mumataja', 'mahala')
('agara', 'phorta')
('taja', 'mahala')
('shahanshaha', 'akabara')
('bamim', 'divara')
('mugala', 'sanrajya')
('sikandara', 'kila')
('sapheda', 'sangamaramara')
('lala', 'balua')
('17vim', 'sadiyom')
('balue', 'patthara')
('shahajada', 'khurrama')
('arjumanda', 'bano')
('upadhi', 'di')
('pahali', 'bara')
('kabhi', 'bujegi|')
('sajaya', 'agara')
('bare', 'aisa')
('agara', 'lane')
('rajya', 'bharata')
('lau', 'bare')
('laimpa', 'jisaki')
('jisaki', 'lau')
('uchcha', 'koti')
('kairina', 'laimpa')
('upara', 'kairina')
('antarika', 'bhaga')
('gayi', 'thim|')
('balua', 'patthara')
('yamuna', 'nadi')
('mana', 'jata')
('mahakavya', 'mahabharata')
('kahani', 'prema')
('lala', 'balue')
('manohara', 'anupata')
('isai', 'vastukala')
('praveshadvara', 'hindu')
('bagiche', 'bicha')
('panchavi', 'manjila')
('hindu', 'muslima')
('muslima', 'isai')
('saghana', 'bagiche')
('kabra', 'saghana')
('manjila', 'puri')
('vastukala', 'vibhinna')
('vibhinna', 'shailiyom')
('shailiyom', 'aka

('banaya', 'uthale')
('uthale', 'pani')
('nadi', 'kinare')
('kinare', 'upari')
('upari', 'tankiyom')
('tankiyom', 'pani')
('sagai', 'samaya')
('leti', 'thim')
('thim', 'bagiche')
('bagiche', 'tatha')
('tatha', 'makabare')
('charom', 'bahati')
('bahati', 'taja़mahala')
('utkatata', 'pradarshita')
('pradarshita', 'karati')
('karati', 'akabara')
('sansara', 'bhara')
('vastu', 'shilpiyom')
('sanrachanatmaka', 'rupa')
('baramade', 'chhata')
('utkirnita', 'buddha')
('baramade', 'chhote')
('daim', 'sahavarti')
('sahavarti', 'bamim')
('bamim', 'sarparaja')
('sarparaja', 'naga')
('naga', 'utkirnita')
('adhika', 'uchcha')
('uchcha', 'stara')
('stara', 'khude')
('khude', 'sankare')
('sankare', 'baramade')
('chara', 'apurna')
('chhata', 'peda़om')
('apurna', 'kaksha')
('matha', 'bamim')
('divara', 'bodhi')
('bodhi', 'vrikshom')
('vrikshom', 'maitreya')
('sata', 'manushi')
('manushi', 'buddha')
('buddha', 'chitrita')
('chitrita', 'chaitya')
('bhaga', 'utkirnita')
('hariti', 'daim')
('devi', 'hariti

('pratibimbita', 'dekha')
('dekha', 'masjida')
('masjida', 'chhata')
('parikalpita', 'emva')
('akabara', 'parikalpita')
('sanrata', 'akabara')
('sansthapaka', 'babara')
('shaili', 'atyanta')
('atyanta', 'uchcha')
('uchcha', 'gunavatta')
('thi|', 'tajamahala')
('shahara', 'samaya')
('samaya', 'mugala')
('rajadhani', 'agara')
('mugala', 'vastukala')
('bhavyata', 'darshate')
('darshate', 'agara')
('babara', 'yamuna')
('phorta|', 'sanrata')
('tatom', 'pahala')
('pahala', 'suvyavasthita')
('suvyavasthita', 'pharasi')
('pharasi', 'baga')
('baga', 'lagavaya')
('lagavaya', 'pautra')
('pautra', 'akabara')
('vishala', 'lala')
('lala', 'kile')
('bhavya', 'parakote')
('mugala', 'shaili')
('samakalina', 'mugala')
('purnataya', 'samakalina')
('imaratem', 'purnataya')
('pratidhvanita', 'hom|')
('hom|', 'sujava')
('jute', 'punah')
('punah', 'prapta')
('bara', 'makabare')
('makabare', 'charom')
('charom', 'bahara')
('saundarya', 'ananda')
('ananda', 'lijie|')
('lijie|', 'madhya')
('arambhika', '17vim')